In [1]:
from functools import wraps
 
def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction
 
@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me!"""
    print("I am the function which needs some decoration to "
          "remove my foul smell")
 
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


In [2]:
a_function_requiring_decoration()

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()


In [3]:
def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated
 
@decorator_name  # 包一层的直接函数名
def func():
    return("Function is running")
 
can_run = True
print(func())
# Output: Function is running
 
can_run = False
print(func())
# Output: Function will not run

Function is running
Function will not run


In [4]:
def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + " was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator  # 返回装饰器
 
@logit()  # 包两层的要调用下
def myfunc1():
    pass
 
myfunc1()
# Output: myfunc1 was called
# 现在一个叫做 out.log 的文件出现了，里面的内容就是上面的字符串
 
@logit(logfile='func2.log')
def myfunc2():
    pass
 
myfunc2()
# Output: myfunc2 was called
# 现在一个叫做 func2.log 的文件出现了，里面的内容就是上面的字符串

myfunc1 was called
myfunc2 was called


In [5]:
import tensorflow as tf

In [13]:
import functools

In [6]:
tf.__version__

'1.15.0'

In [7]:
tf.get_default_graph().get_name_scope()

''

In [11]:
with tf.name_scope('V1'):  
    # a1 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))  
    a2 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')  
with tf.name_scope('V2'):  
    # a3 = tf.get_variable(name='a1', shape=[1], initializer=tf.constant_initializer(1))  
    a4 = tf.Variable(tf.random_normal(shape=[2,3], mean=0, stddev=1), name='a2')  
    print(tf.get_default_graph().get_name_scope())
    
with tf.Session() as sess:  
    sess.run(tf.initialize_all_variables())  
    # print a1.name  
    print(a2.name)  
    # print a3.name  
    print(a4.name)  

V2_1
V1_1/a2:0
V2_1/a2:0


In [10]:
tf.get_default_graph().get_name_scope()

''

In [12]:
class Meta(type):  
   
    record_cls = []

    def __new__(cls, name, bases, attrs):
        new_cls = type.__new__(cls, name, bases, attrs)
        Meta.record_cls.append(new_cls)
        return new_cls
    
    def __init__(cls, name, bases, namespace, **kwargs):
        super().__init__(name, bases, namespace, **kwargs)
        if not hasattr(cls, 'registory'):  # build only in the first subclass
            # this is the base class
            cls.registory = {}
        else:
            # this is the subclass
            cls.registory[name.lower()] = cls
            
    def create(cls, class_name, *args, **kwargs):
        return cls.registory[class_name.lower()](*args, **kwargs)


In [79]:
class Module(object, metaclass=Meta):
    
    def __init__(self, name, **kwargs):
        self.name = name+'qq'
        
        self.parent_name = tf.get_default_graph().get_name_scope()
        if self.parent_name:
            self.total_name = self.parent_name + '/' + self.name + '/'
        else:
            self.total_name = self.name + '/'
        self._ns_call_count = 0

        self.name_scope = tf.name_scope(self.total_name)
        self.restore_variables = {}
        self.summaries = []
        self.updates = []
        print('====',self.name,name,self._ns_call_count)

    @staticmethod
    def with_name_scope(func):
        
        @functools.wraps(func)
        def inner(self, *args, **kwargs):
            parent_name = tf.get_default_graph().get_name_scope()
            name = self.name + '_-_' + str(self._ns_call_count) if self._ns_call_count else self.name
            if parent_name:
                total_name = parent_name + '/' + name + '/'
            else:
                total_name = name + '/'
            self._ns_call_count += 1
            print('inner',self._ns_call_count,total_name,parent_name,name)
            with tf.name_scope(total_name):
                return func(self, *args, **kwargs)

        return inner

    def _get_recursive_list(self, key, seen):
        ret = []

        if self in seen:
            return ret
        seen.add(self)
        ret.extend(getattr(self, key))

        def _extend(val):
            if isinstance(val, Module):
                ret.extend(val._get_recursive_list(key, seen))
            if isinstance(val, tf.keras.layers.Layer):
                if not (val in seen):
                    ret.extend(getattr(val, key))
                seen.add(val)

        for attr in dir(self):
            val = getattr(self, attr)
            if isinstance(val, list):
                for v in val:
                    _extend(v)
            elif isinstance(val, dict):
                for k, v in val.items():
                    _extend(v)
            else:
                _extend(val)

        return ret

    def get_updates(self, seen=None):
        if not isinstance(seen, set):
            seen = set()
        return self._get_recursive_list('updates', seen)

In [80]:
class Dense(Module):

    def __init__(self, name,):
        super(Dense, self).__init__(name)
        with self.name_scope:
            print('+',name,tf.get_default_graph().get_name_scope(),self.name_scope)
            self.w = tf.Variable(
                tf.random_normal(shape=[2,3], mean=0, stddev=1),
                name='w', 
                dtype=tf.float32)
            self.b = tf.Variable(
                tf.random_normal(shape=[2,3], mean=0, stddev=1),
                name='b',
                dtype=tf.float32)

    @Module.with_name_scope
    def __call__(self, training=False):
        print(tf.get_default_graph().get_name_scope())
        return training

In [81]:
class MLP(Module):

    def __init__(self, name):
        super(MLP, self).__init__(name)
        with self.name_scope:
            print('*',name,tf.get_default_graph().get_name_scope(),self.name_scope)
            self.layers = []
            for i in range(4):
                self.layers.append(
                    Dense(
                        'dense_%d' % (i,),
                        )
                    )

    @Module.with_name_scope
    def __call__(self,training=False):
        print(tf.get_default_graph().get_name_scope())
        return 1

In [82]:
a = MLP('mlp')

==== mlpqq mlp 0
* mlp mlpqq <tensorflow.python.framework.ops.name_scope object at 0x0000029EFD86BA90>
==== dense_0qq dense_0 0
+ dense_0 mlpqq/dense_0qq <tensorflow.python.framework.ops.name_scope object at 0x0000029EFD86B940>
==== dense_1qq dense_1 0
+ dense_1 mlpqq/dense_1qq <tensorflow.python.framework.ops.name_scope object at 0x0000029EFD86B320>
==== dense_2qq dense_2 0
+ dense_2 mlpqq/dense_2qq <tensorflow.python.framework.ops.name_scope object at 0x0000029EFD889898>
==== dense_3qq dense_3 0
+ dense_3 mlpqq/dense_3qq <tensorflow.python.framework.ops.name_scope object at 0x0000029EFD87CE48>


In [83]:
a()


inner 1 mlpqq/  mlpqq
mlpqq


1

In [84]:
a.layers[3]()

inner 1 dense_3qq/  dense_3qq
dense_3qq


False

In [21]:
a.name

'mlp'

In [35]:
tf.global_variables()

[<tf.Variable 'V1/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'V2/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'V1_1/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'V2_1/a2:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_0/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_0/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_1/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_1/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_2/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_2/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_3/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlp/dense_3/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_0qq/b:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_1qq/w:0' shape=(2, 3) dtype=float32_ref>,
 <tf.Variable 'mlpqq/dense_1qq/b:0' shape

In [32]:
a()

mlpqq


1